<a href="https://colab.research.google.com/github/cedkim/deeplearning-courseenv/blob/master/4-tensorflow-audiobooks-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical example: Audiobooks

## Import the necessary libraries and download the files

In [2]:
import numpy as np
from sklearn import preprocessing
!wget -O audiobooks_data.csv "https://onedrive.live.com/download?cid=22B0F983382BA466&resid=22B0F983382BA466%21299394&authkey=AKsawparTh_7fTc"

--2020-08-05 09:43:12--  https://onedrive.live.com/download?cid=22B0F983382BA466&resid=22B0F983382BA466%21299394&authkey=AKsawparTh_7fTc
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://hhf3tw.am.files.1drv.com/y4mfbmfXg5p1qoZTj0AOWKqpHx7XOfefm3SJx5bMyjZ8uWIAKadB26epNe7b2sR58BRk-nR8ROGPRr_4pTIrOF1DGA5N97gXaZj6FTimNeQalgn8g7mNc724ILoNG4kNJUks6YzXxXu9Q_ickPdOPzkfuK896u4x5d_gh-aPNj0q9kRrUleG6SN-TE1w63_fCKNyK_Ua3upfGpV-m4qdwsrNA/1.1%20Audiobooks_data.csv.csv?download&psid=1 [following]
--2020-08-05 09:43:13--  https://hhf3tw.am.files.1drv.com/y4mfbmfXg5p1qoZTj0AOWKqpHx7XOfefm3SJx5bMyjZ8uWIAKadB26epNe7b2sR58BRk-nR8ROGPRr_4pTIrOF1DGA5N97gXaZj6FTimNeQalgn8g7mNc724ILoNG4kNJUks6YzXxXu9Q_ickPdOPzkfuK896u4x5d_gh-aPNj0q9kRrUleG6SN-TE1w63_fCKNyK_Ua3upfGpV-m4qdwsrNA/1.1%20Audiobooks_data.csv.csv?download&psid=1
Resolving hhf3tw.am.file

## Preprocess the data

### Extract the data from the csv

In [3]:
raw_csv_data = np.loadtxt('audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [4]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [7]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
test_targets = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1801.0 3579 0.5032131880413523
118.28579839065503 447 0.2646214729097428
118.28579839065503 448 0.2640307999791407


### Save the three datasets in *.npz